In [1]:
# データのインポート
import pandas as pd

filepath_train = "./data/sales_train.csv"
filepath_test = "./data/test.csv"

df_train = pd.read_csv(filepath_or_buffer = filepath_train)
df_test = pd.read_csv(filepath_or_buffer = filepath_test)

df_train.shape, df_test.shape

((2935849, 6), (214200, 3))

In [2]:
# [date]列を"object --> datetime"へ型変換
from datetime import datetime
df_train["date"] = df_train["date"].apply(lambda x:datetime.strptime(x, '%d.%m.%Y'))

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   date_block_num  int64         
 2   shop_id         int64         
 3   item_id         int64         
 4   item_price      float64       
 5   item_cnt_day    float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 134.4 MB


In [4]:
df_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


### 案１：過去のデータ全ての売り上げを対象に重みづけを行うという方法

1. 店舗ごとの総売上を合算
2. 売上額を降順でソート
3. 全体の１になるように店舗数(60)で重み付けを実施。

In [5]:
import pandas as pd
import numpy as np

class f_engineering():

    def __init__(self, df_train, df_test):
        self.df_train = df_train
        self.df_test = df_test

    # 店舗ごとの販売合計数を集計
    def _shop_total(self):

        shop_total = self.df_train.groupby("shop_id").sum()
        shop_total = shop_total.drop(["date_block_num", "item_id", "item_price"], axis = 1)
        shop_total.rename(columns = {"item_cnt_day" : "item_cnt_total"}, inplace = True)

        return shop_total
    
    # 店舗ごとの重みを計算
    def _culc_w_shop(self):
        shop_total = self._shop_total()
        
        ## 販売数合計
        item_total = np.sum(shop_total.values)

        ## 重みのリスト(shop数)
        w = np.zeros(shop_total.shape[0])

         ## 重み　＝　各店舗 ÷ 販売数合計 × 100
        for i in range(shop_total.shape[0]):
            w[i] = shop_total.iloc[i].values / item_total * 100

        shop_total["W"] = w
        
        return shop_total

    def output_w(self):
        
        shop_total = self._culc_w_shop()

        # df_testに重み用の空の箱を作る
        w_test = np.zeros(len(self.df_test))
        self.df_test["W"] = w_test

        # テストデータが持つショップIDと合致するショップIDの重みをdf_test["W"]に格納する。
        for i in range(self.df_test.shape[0]):
            for j in range(shop_total.shape[0]):
                if self.df_test["shop_id"][i] != shop_total.index[j]:
                    pass
                else:
                    self.df_test["W"][i] = shop_total["W"][j]

        # df_test["W"]の合計値を計算
        w_total = np.sum(self.df_test.W)

        # df_test["W"]の合計値を元に、それぞれの重みを更新。 --> 全体で１になるように。
        for i in range(df_test.shape[0]):
            self.df_test["W"][i] = self.df_test["W"][i] / w_total

        return self.df_test

### 案２：過去の11月時の売り上げを対象に重みづけを行うという方法

In [6]:
# 
df_train['month'] = df_train['date'].dt.to_period('M')
df_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month
0,2013-01-02,0,59,22154,999.00,1.0,2013-01
1,2013-01-03,0,25,2552,899.00,1.0,2013-01
2,2013-01-05,0,25,2552,899.00,-1.0,2013-01
3,2013-01-06,0,25,2554,1709.05,1.0,2013-01
4,2013-01-15,0,25,2555,1099.00,1.0,2013-01
...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0,2015-10
2935845,2015-10-09,33,25,7460,299.00,1.0,2015-10
2935846,2015-10-14,33,25,7459,349.00,1.0,2015-10
2935847,2015-10-22,33,25,7440,299.00,1.0,2015-10


In [7]:
df_train_11 = df_train.query('date_block_num == 10 | date_block_num == 22')

In [8]:
shop_total_11 = df_train_11.groupby("shop_id").sum()
shop_total_11 = shop_total_11.drop(["date_block_num", "item_id", "item_price"], axis = 1)

/var/folders/6x/0whm9kbs4bg6p6xq6w7vsm0r0000gn/T/ipykernel_33997/3610535706.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  shop_total_11 = df_train_11.groupby("shop_id").sum()


In [9]:
total_train = np.sum(shop_total_11.item_cnt_day.values)

In [10]:
w = np.zeros(len(shop_total_11))

In [11]:
## 重み　＝　各店舗 ÷ 販売数合計 × 100
for i in range(shop_total_11.shape[0]):
    w[i] = shop_total_11.iloc[i,:].values / total_train * 100

In [45]:
shop_total_11["W"] = w
shop_total_11["shop_id"] = shop_total_11.index
shop_total_11

,item_cnt_day,W,shop_id
shop_id,,,
2,2054.0,0.828714,2
3,2033.0,0.820241,3
4,2857.0,1.152695,4
5,2835.0,1.143819,5
6,7088.0,2.859748,6
7,4565.0,1.841810,7
10,1776.0,0.716551,10
12,6607.0,2.665682,12
13,1052.0,0.424443,13


In [35]:
w_test = np.zeros(len(df_test))

Index(['item_cnt_day', 'W'], dtype='object')

In [36]:
df_train["item_cnt_day"][]

1.0

In [43]:
type(w_test)
inlist = shop_total_11.index
list(inlist)

[2,
 3,
 4,
 5,
 6,
 7,
 10,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 21,
 22,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59]

In [49]:
for i in range(df_test.shape[0]):
    for j in list(inlist):
        if df_test["shop_id"][i] != shop_total_11["shop_id"][j]:
            pass
        else:
            w_test[i] = shop_total_11.W[j]

In [48]:
w_test

array([1.14381854, 1.14381854, 1.14381854, ..., 1.16036053, 1.16036053,
       1.16036053])

In [ ]:
# df_testに重み用の空の箱を作る



# テストデータが持つショップIDと合致するショップIDの重みをdf_test["W"]に格納する。


# df_test["W"]の合計値を計算
w_total = np.sum(df_test.W)

# df_test["W"]の合計値を元に、それぞれの重みを更新。 --> 全体で１になるように。
for i in range(df_test.shape[0]):
    df_test["W"][i] /= w_total

/var/folders/6x/0whm9kbs4bg6p6xq6w7vsm0r0000gn/T/ipykernel_22896/3683936282.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["W"][i] = shop_total_11["W"][j]


KeyError: 1

In [ ]:
shop_total_11

,item_cnt_day,W
shop_id,,
2,2054.0,0.828714
3,2033.0,0.820241
4,2857.0,1.152695
5,2835.0,1.143819
6,7088.0,2.859748
7,4565.0,1.841810
10,1776.0,0.716551
12,6607.0,2.665682
13,1052.0,0.424443


### 案３：(売上数, 〇〇)で散布図を描画し、クラスタリングをして、重みづけをつけるという方法